In [1]:
import os
import glob
import json
import re
import pickle

In [2]:
def getTR(path):
    with open(path, "r") as f:
        jsonObj = json.load(f)
    return jsonObj["RepetitionTime"]

In [12]:
base_path = "../datasets/ds002799/derivatives/fmriprep/"

In [13]:
subs = glob.glob(base_path+"/sub-*/")
print(subs[0])
reg_sub = re.compile(r"sub-\d+")
sub_ids = [reg_sub.search(i)[0] for i in subs]

In [ ]:
time_series = None
if os.path.exists("time_series.pkl"):
    with open("time_series.pkl", "rb") as f:
        time_series = pickle.load(f)
def getTimeSeries(sub, op, run):
    if time_series is None:
        return ""
    return time_series[sub][op][run]["time_series"]

In [10]:
datasets = {
    sub : {
        "ses-preop" : {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0])
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-preop/func/*MNI152*preproc_bold.nii.gz"))
        },
        "ses-postop": {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0])
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-postop/func/*MNI152*preproc_bold.nii.gz"))
        }
        } for sub in sub_ids
    }

In [ ]:
data = {
    sub : {
        "ses-preop" : {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-preop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0]),
                "time_series" : getTimeSeries(sub, "ses-preop", f"run-{i+1:0>2d}")
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-preop/func/*MNI152*preproc_bold.nii.gz"))
        },
        "ses-postop": {
            "run-"+f"{i+1:0>2d}" : {
                "bold" : v,
                "mask" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*mask.nii.gz")[0],
                "confounds" : glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*confounds_regressors.tsv")[0],
                "TR" : getTR(glob.glob(base_path + sub + "/ses-postop/func/*run-" + f"{i+1:0>2d}" + "*MNI152*preproc_bold.json")[0]),
                "time_series" : getTimeSeries(sub, "ses-postop", f"run-{i+1:0>2d}")
            } for i,v in enumerate(glob.glob(base_path + sub + "/ses-postop/func/*MNI152*preproc_bold.nii.gz"))
        }
        } for sub in sub_ids
    }

In [11]:
if time_series:
    with open("time_series.pkl", "wb") as f:
        pickle.dump(data, f)
with open("datasets.json", "w") as f:
    json.dump(datasets, f)

In [3]:
cmp_base_path = "../datasets/ds000031_R1.0.4/derivatives/fmriprep"

In [4]:
subs = glob.glob(cmp_base_path+"/sub-*/")
print(subs[0])
reg_sub = re.compile(r"sub-\d+")
sub_ids = [reg_sub.search(i)[0] for i in subs]

../datasets/ds000031_R1.0.4/derivatives/fmriprep\sub-01\


In [20]:
datasets_cmp = {}
for sub in sub_ids:
    datasets_cmp[sub] = {}
    for ses in glob.glob(f"{cmp_base_path}/{sub}/ses*"):
        ses = re.search(r"ses-\d+", ses)[0]
        datasets_cmp[sub][ses] = {}
        for i,v in enumerate(glob.glob(f"{cmp_base_path}/{sub}/{ses}/func/*rest*MNI152*preproc.nii.gz")):
            run = "run-"+f"{i+1:0>3d}"
            datasets_cmp[sub][ses][run] = {
                "bold" : v,
                "confounds" : glob.glob(f"{cmp_base_path}/{sub}/{ses}/func/*rest*{run}*confounds.tsv")[0],
                "mask" : glob.glob(f"{cmp_base_path}/{sub}/{ses}/func/*rest*{run}*MNI152*mask.nii.gz")[0],
                "TR" : 1.16
            }

with open("datasets_cmp.json", "w") as f:
    json.dump(datasets_cmp, f)